#  SQL Project : E-commerce RFM Segmentation

This project demonstrates how to perform **RFM (Recency, Frequency, Monetary) segmentation** using **SQLite** in a Jupyter Notebook.

We will use real-world online retail transaction data to segment customers based on their purchasing behavior. This is a common technique in **marketing analytics** and **customer retention strategies**.


In [1]:
import pandas as pd
import sqlite3

##  Step 1: Load Dataset and Push to SQLite

We begin by loading the e-commerce dataset, converting the date column, and pushing the data into a local SQLite database. This makes it easy to perform SQL-based analysis inside Python.


In [2]:
df = pd.read_csv("Online Retail.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Total_Amount,CustomerID,Country
0,538521,21754,HOME BUILDING BLOCK WORD,3,13-12-2010,5.95,17.85,14180,United Kingdom
1,538521,21755,LOVE BUILDING BLOCK WORD,3,13-12-2010,5.95,17.85,14180,United Kingdom
2,538521,22072,RED RETROSPOT TEA CUP AND SAUCER,8,13-12-2010,3.75,30.00,14180,United Kingdom
3,538521,22846,BREAD BIN DINER STYLE RED,1,13-12-2010,16.95,16.95,14180,United Kingdom
4,538521,22849,BREAD BIN DINER STYLE MINT,1,13-12-2010,16.95,16.95,14180,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337263 entries, 0 to 337262
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   InvoiceNo     337263 non-null  int64  
 1   StockCode     337263 non-null  object 
 2   Description   337263 non-null  object 
 3   Quantity      337263 non-null  int64  
 4   InvoiceDate   337263 non-null  object 
 5   UnitPrice     337263 non-null  float64
 6   Total_Amount  337263 non-null  float64
 7   CustomerID    337263 non-null  int64  
 8   Country       337263 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 23.2+ MB


In [4]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], dayfirst=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337263 entries, 0 to 337262
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     337263 non-null  int64         
 1   StockCode     337263 non-null  object        
 2   Description   337263 non-null  object        
 3   Quantity      337263 non-null  int64         
 4   InvoiceDate   337263 non-null  datetime64[ns]
 5   UnitPrice     337263 non-null  float64       
 6   Total_Amount  337263 non-null  float64       
 7   CustomerID    337263 non-null  int64         
 8   Country       337263 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 23.2+ MB


In [6]:
conn = sqlite3.connect("ecommerce_rfm.db")
df.to_sql("transactions", conn, if_exists="replace", index=False)

337263

##  Step 2: Get Latest Invoice Date

RFM analysis depends on a reference date. We use the **latest transaction date** in the dataset as our reference to calculate Recency (days since last purchase).


In [7]:
query = "SELECT MAX(InvoiceDate) AS latest_date FROM transactions;"
pd.read_sql_query(query, conn)

,latest_date
0,2011-12-09 00:00:00


##  Step 3: Calculate Recency, Frequency, Monetary Values

We calculate RFM metrics per customer:
- **Recency**: Days since the last purchase
- **Frequency**: Number of unique orders
- **Monetary**: Total purchase amount

These metrics are the foundation of customer segmentation.


In [8]:
query = """
SELECT 
    CustomerID,
    ROUND(JULIANDAY('2011-12-10') - JULIANDAY(MAX(InvoiceDate)), 0) AS Recency,
    COUNT(DISTINCT InvoiceNo) AS Frequency,
    ROUND(SUM(Total_Amount), 2) AS Monetary
FROM transactions
WHERE CustomerID IS NOT NULL
GROUP BY CustomerID;
"""

rfm = pd.read_sql_query(query, conn)
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346,326.0,1,77183.60
1,12347,40.0,7,4310.00
2,12348,76.0,4,1797.24
3,12350,311.0,1,334.40
4,12352,73.0,8,2506.04


##  Step 4: Score Each Metric from 1 to 5

Each customer is scored on Recency, Frequency, and Monetary metrics using **quantile binning**. This creates a consistent scale from 1 (lowest) to 5 (highest).
- Lower Recency = Higher score (recent buyers)
- Higher Frequency = Higher score (loyal buyers)
- Higher Monetary = Higher score (big spenders)


In [9]:
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1]).astype(int)
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm['M_score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]).astype(int)

##  Step 5: Create RFM Score

We now combine the individual R, F, and M scores into a single RFM string (e.g., 514). This score helps us cluster customers into meaningful behavioral groups.


In [10]:
rfm['RFM_Score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

##  Step 6: Define Customer Segments

Based on RFM Score, we assign labels to customer groups:
- **555** → Champions
- **R=5** → Recent Customers
- **F=5** → Loyal Customers
- **M=5** → Big Spenders
- **111** → Lost
- **R=1** → At Risk
- Else → Others

These segments help target campaigns effectively.


In [11]:
def assign_segment(score):
    if score == '555':
        return 'Champions'
    elif score[0] == '5':
        return 'Recent Customers'
    elif score[1] == '5':
        return 'Loyal Customers'
    elif score[2] == '5':
        return 'Big Spenders'
    elif score == '111':
        return 'Lost'
    elif score[0] == '1':
        return 'At Risk'
    else:
        return 'Others'

rfm['Segment'] = rfm['RFM_Score'].apply(assign_segment)
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Score,Segment
0,12346,326.0,1,77183.60,1,1,5,115,Big Spenders
1,12347,40.0,7,4310.00,5,5,5,555,Champions
2,12348,76.0,4,1797.24,3,4,4,344,Others
3,12350,311.0,1,334.40,1,1,2,112,At Risk
4,12352,73.0,8,2506.04,3,5,5,355,Loyal Customers


In [18]:
rfm.to_csv("Final_RFM_Segmentation.csv", index=False)